In [2]:
from importlib import reload , import_module
import MQ
reload(MQ)

from MQ import list_uuid_folders_by_creation,resend_failed_webhooks

todos = list_uuid_folders_by_creation()   
for f in todos:
    print(f[0])
    await resend_failed_webhooks(f[0], "https://www.example.com/oauth2/MQ")

In [4]:
from importlib import reload , import_module
from fastapi.testclient import TestClient

import MQ
reload(MQ)

import redis  #.asyncio as redis
r = redis.Redis(host="localhost", port=6379, db=9,decode_responses=True)
# ----------- Optional Test ----------- #
def run_test():
    client = TestClient(MQ.app)

    print("🚀 Running test with status enum...")

    
    res = client.post("/MQ/send", json={
        "topic": "topic.abc",
        "payload": {"cmd1": "convert"},
        "target": "worker1",
        "resource": "tdd",
        "priority": 10
    })

    res = client.post("/MQ/send", json={
        "topic": "topic.abc",
        "payload": {"cmd2": "convert"},
        "target": "worker1",
        "resource": "tdd",
        "priority": 10
    })
    
    assert res.status_code == 200
    uuid = res.json()["uuid"]
    # new_priority 优先级修改
    assert client.post(f"/MQ/priority", json={
                                            "uuid": uuid, 
                                            "target": "worker1",
                                            "new_priority":1
                                        }).status_code == 200

    nxt = client.get("/MQ/worker1/poll").json()
    print(uuid,'\r\n',nxt)
    
    # cancel mq 取消pendding的MQ
    assert client.post(f"/MQ/cancel", json={
                                            "uuid": uuid, 
                                            "target": "worker1"
                                        }).status_code == 200

    nxt = client.get("/MQ/worker1/poll").json()
    print(uuid,'\r\n',nxt)
    uuid = nxt["uuid"]
    #by id
    assert client.get(f"/MQ/worker1/poll/{uuid}").json()["uuid"] == uuid

    #ack
    assert client.post(f"/MQ/worker1/ack/{uuid}").status_code == 200

    #processing
    assert client.post(f"/MQ/worker1/status", json={"uuid": uuid, "status": "Processing"}).status_code == 200

    #complete
    assert client.post(f"/MQ/worker1/status", json={"uuid": uuid, "status": "Completed", "detail": {"out": "done.mp4"}}).status_code == 200
    
    #完成后队列会清理 404
    nxt = client.get(f"/worker1/pool/{uuid}")
    print(uuid,'\r\n',nxt)

    #
    assert client.get("/MQ/worker1/stats/topic.abc?status=Completed").json()["count"] == 0
    print("✅ Test passed!")

    client.get("/MQ/webhook").json()

#r.flushdb()
run_test()    

🚀 Running test with status enum...
Webhook 发送成功: 200
Webhook 发送成功: 200
934f6198-140d-4f6c-b978-4e23920ca684 
 {'uuid': '934f6198-140d-4f6c-b978-4e23920ca684', 'topic': 'topic.abc', 'payload': "{'cmd2': 'convert'}", 'target': 'worker1', 'resource': 'tdd', 'priority': '1', 'timestamp': '2025-07-17T04:33:52.540708+00:00', 'status': 'Pending'}
Webhook 发送成功: 200
消息 934f6198-140d-4f6c-b978-4e23920ca684 因状态 Cancelled 被删除
934f6198-140d-4f6c-b978-4e23920ca684 
 {'uuid': 'f319c9ee-5033-44bf-817d-0e351e7cd025', 'topic': 'topic.abc', 'payload': "{'cmd1': 'convert'}", 'target': 'worker1', 'resource': 'tdd', 'priority': '10', 'timestamp': '2025-07-17T04:33:52.356192+00:00', 'status': 'Pending'}
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
消息 f319c9ee-5033-44bf-817d-0e351e7cd025 因状态 Completed 被删除
f319c9ee-5033-44bf-817d-0e351e7cd025 
 <Response [404 Not Found]>
✅ Test passed!
913a4063-7d39-4ffe-a2b0-7af0d08b5e85
✅ Webhook 发送成功: webhook_failures/913a4063-7d39-4ffe-a2b0-7af0d08b5e85/20250717T0

In [95]:
import concurrent.futures
from fastapi.testclient import TestClient
import random
import time
from importlib import reload, import_module

import MQ
reload(MQ)
client = TestClient(MQ.app)

# 清空 Redis
MQ.r.flushdb()

NUM_CLIENTS =10
TOPIC = "stress.topic"
TARGET = "worker1"

def simulate_client(i):
    # query is empty
    res = client.get(f"/MQ/{TARGET}/poll")
    if res.status_code == 200 and not res.json():
        print("✅ Queue is empty after all messages processed")

    # Send message
    res = client.post("/MQ/send", json={
        "topic": TOPIC,
        "payload": {"cmd": f"process-{i}"},
        "target": TARGET,
        "resource": "TDD",
        "priority": random.randint(1, 10)
    })
    if res.status_code != 200:
        return f"Client-{i} send failed"

    uuid = res.json()["uuid"]

    # Poll
    msg = client.get(f"/MQ/{TARGET}/poll/{uuid}").json()
    if not msg or msg["uuid"] != uuid:
        return f"Client-{i} poll mismatch"

    # ACK
    res = client.post(f"/MQ/{TARGET}/ack/{uuid}")
    if res.status_code != 200:
        return f"Client-{i} ack failed"

    # Status updates
    for status in ["Started", "Processing", "Completed"]:
        res = client.post(f"/MQ/{TARGET}/status", json={
            "uuid": uuid,
            "status": status,
            "detail": {"step": status}
        })
        if res.status_code != 200:
            return f"Client-{i} update {status} failed"
    
    return f"Client-{i} ✅ success"

def run_concurrent_test():
    print(f"🚀 Starting stress test with {NUM_CLIENTS} clients...")
    start = time.time()

    with concurrent.futures.ThreadPoolExecutor(max_workers=NUM_CLIENTS) as executor:
        results = list(executor.map(simulate_client, range(NUM_CLIENTS)))

    for r in results:
        print(r)

    print(f"⏱️ Test completed in {time.time() - start:.2f}s")

run_concurrent_test()


🚀 Starting stress test with 10 clients...
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
✅ Queue is empty after all messages processed
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhook 发送成功: 200
Webhoo

In [ ]:
from fastapi.testclient import TestClient
from importlib import reload
import MQ

reload(MQ)  # 确保代码更新
client = TestClient(MQ.app)
r = MQ.r  # 使用你 MQ 模块中的 Redis 实例

def test_poll_ack_complete_cycle(n=5):
    print(f"\n🚀 Starting test: sending {n} messages to worker1")
    #r.flushdb()

    uuids = []

    # Step 1: Send n messages
    for i in range(n):
        res = client.post("/MQ/send", json={
            "topic": f"topic.{i}",
            "payload": {"cmd": f"task-{i}"},
            "target": "worker1",
            "resource": "tdd",
            "priority": 10 + i
        })
        assert res.status_code == 200
        uuid = res.json()["uuid"]
        uuids.append(uuid)
        print(f"📤 Sent message {i+1}: {uuid}")

    # Step 2: Start polling and processing
    for i in range(n-1):
        print(f"\n🔁 Iteration {i+1}: Poll and process message")

        # First poll — should get the next message
        poll1 = client.get("/MQ/worker1/poll")
        assert poll1.status_code == 200
        msg1 = poll1.json()
        assert msg1 is not None
        curr_uuid = msg1["uuid"]
        print(f"📥 First poll got UUID: {curr_uuid}")

        # Second poll — should be the same (since not acked)
        poll2 = client.get("/MQ/worker1/poll")
        msg2 = poll2.json()
        assert msg2["uuid"] == curr_uuid
        print(f"📥 Second poll same UUID: {msg2['uuid']}")

        # Acknowledge the message
        ack = client.post(f"/MQ/worker1/ack/{curr_uuid}")
        assert ack.status_code == 200
        print(f"✅ Acknowledged UUID: {curr_uuid}")

        # # Mark as completed
        # status = client.post("/MQ/worker1/status", json={
        #     "uuid": curr_uuid,
        #     "status": "Completed",
        #     "detail": {"output": f"done-{i}.txt"}
        # })
        # assert status.status_code == 200
        # print(f"✅ Marked as Completed: {curr_uuid}")

    # Step 3: Final poll — should return null (no more messages)
    final = client.get("/MQ/worker1/poll")
    assert final.status_code == 200
    # assert final.json() in (None, {})  # Depends on implementation
    print("\n✅ All messages processed. Queue is now empty.")

# Run the test
test_poll_ack_complete_cycle(n=5)
client.get("/MQ/worker1/stats").json()